In [1]:
import torch
import os
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import json



from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score
from sklearn.manifold import TSNE
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from spdnetwork.optimizers import  MixOptimizer 
from spdnetwork.nn import LogEig
from Utils import get_fold_of_data
from DatasetManagement import DatasetManagement
from Models import   Contrastive_CB3

device = 'cuda' if torch.cuda.is_available() else 'cpu'

%load_ext autoreload
%autoreload 2

/opt/conda/lib/python3.10/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 101: invalid device ordinal (Triggered internally at /opt/conda/conda-bld/pytorch_1678411187366/work/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [1]:
96 // 4

24

In [2]:




'''


'''

def generate_vectors(info_run): 
    
    target_shape = info_run['target_shape']
    sequence_embedding_features = info_run['sequence_embedding_features']
    weights_path = info_run['weights_path']

    model = Contrastive_CB3(device, target_shape, sequence_embedding_features, 'contrastive')
    model.load_state_dict(torch.load(weights_path, map_location=torch.device('cpu')))
    
    
    # ===================================================================================================
    
    
    #LOAD DATA
    data_loader = info_run['data']
    
    # ===================================================================================================    
    #OBTENER TODAS LAS ACTIVACIONES DE UN FOWARD PASS

    def get_activation(name):
        def hook(model, input, output):
            activations[name] = output
        return hook

    activations = {}
    all_activations_t2 = []
    all_activations_adc = []
    all_activations_bval = []
    all_labels = [] 

    # Registro el hook en la capa 17
    model.t2_conv_branch[17].register_forward_hook(get_activation('t2_conv_branch'))
    model.adc_conv_branch[17].register_forward_hook(get_activation('adc_conv_branch'))
    model.bval_conv_branch[17].register_forward_hook(get_activation('bval_conv_branch'))

    model.eval()
    model.to(device)

    with torch.no_grad():
        for i, data in enumerate(data_loader, 0):
            inputs, labels = data
            for j in range(len(inputs)):
                inputs[j] = inputs[j].to(device).type(torch.float)
            labels = labels.to(device)
            
            outputs = model(inputs)
            
            all_labels.append(labels)
            
            all_activations_t2.append(activations['t2_conv_branch'])
            all_activations_adc.append(activations['adc_conv_branch'])
            all_activations_bval.append(activations['bval_conv_branch'])
            

    concatenated_activations = []

    for t2_act, adc_act, bval_act in zip(all_activations_t2, all_activations_adc, all_activations_bval):
        concatenated = torch.cat([t2_act, adc_act, bval_act], dim=1)
        concatenated_activations.append(concatenated)
        
    #print(concatenated_activations[7].shape) # torch.Size([28, 96, 6, 8, 8])
    print(len(concatenated_activations)) # 8


    all_labels_tensor = torch.cat(all_labels, dim=0)
            
    print(all_labels_tensor.shape)  # Debería imprimir el tamaño total de todos los labels

    # ===================================================================================================    


    #GENERAR LAS MATRICES SPD'S
    def get_spd(acts, type="gramm"):
        
        vect_acts = acts
        # h, w, d = acts.shape
        # vect_acts = acts.reshape(h * w, d)
        if type == "gramm":
            spd = vect_acts.T @ vect_acts
        elif type == "corr":
            spd = np.corrcoef(vect_acts.T)
        elif type == "cov":
            spd = np.cov(vect_acts.T)
        return spd

    all_spds = []

    # Recorro los 8 lotes de 32
    for batch in concatenated_activations:  # batch = [32, 96, 6, 8, 8]
        batch_size, channels, depth, height, width = batch.shape
        
        # Iterar sobre cada muestra en el lote
        for i in range(batch_size):
            sample = batch[i]  # sample = [96, 6, 8, 8]
            #print(f"Sample shape: {sample.shape}")
            
            sample_reshape = sample.permute(1, 2, 3, 0).reshape(-1, channels) # sample_reshape = torch.Size([384, 96])
            # print(sample_reshape.shape)
            
            activation_np = sample_reshape.cpu().numpy()
            
            spd_matrix = get_spd(activation_np, type="gramm")
            
            all_spds.append(spd_matrix)
                      


    all_spds_tensor = torch.tensor(all_spds)
    print(all_spds_tensor.shape)
    print(all_spds_tensor[0].shape)
    
    # ===================================================================================================    

    #MAPEAR AL PLANO TANGENTE
    # Función para obtener las matrices SPD reconstruidas

    num_spds, x, y = all_spds_tensor.shape

    reconstructed_spds = torch.zeros(num_spds, x, y)

    for i in range(num_spds):
        spd = all_spds_tensor[i]
        eigenvalues, eigenvectors = torch.linalg.eigh(spd)
        
        epsilon = 1e-5
        log_eigenvalues = torch.log(eigenvalues.clamp(min=epsilon))
        
        reconstructed_spd = eigenvectors @ torch.diag(log_eigenvalues) @ eigenvectors.T
        
        reconstructed_spds[i] = reconstructed_spd
        


    reconstructed_spds.shape  #reconstructed_spds = torch.Size([252, 96, 96])
    
    
    # ===================================================================================================    


    n_samples, _, _ = reconstructed_spds.shape
    
    upper_triangular_flat = []
    
    for i in range(n_samples):
        spd = reconstructed_spds[i]
        
        upper_triangular = torch.masked_select(torch.triu(spd, diagonal=1), torch.triu(torch.ones_like(spd), diagonal=1).bool())
        
        upper_triangular_flat.append(upper_triangular)
    
    upper_triangular_flat_tensor = torch.stack(upper_triangular_flat)
    

#upper_triangular_flat_tensor.shape#upper_triangular_flat = [252, 4560]

    
    
    embbedings = upper_triangular_flat_tensor
    final_labels = all_labels_tensor
    
    
    
    return embbedings, final_labels


    



In [29]:
    #RUN
        
    #DEFINO LOS DATOS
for k in range(5):
    
    
        x_train, x_validation ,y_train, y_validation, ids_train, ids_val, indexdes = get_fold_of_data(k, 0.80) #CAMBIAR
        
        print(f"X_train shape: {x_train.shape}")
        print(f"Y_train shape: {y_train.shape}")
        print(f"X_val shape: {x_validation.shape}")
        print(f"Y_val shape: {y_validation.shape}")

        val_data = DatasetManagement(x_validation, y_validation)
        train_data = DatasetManagement(x_train, y_train)

        val_loader = torch.utils.data.DataLoader(
            dataset = val_data,
            shuffle = False,
            batch_size = 32,
            pin_memory=False
        )

        train_loader = torch.utils.data.DataLoader(
            dataset = train_data,
            shuffle = False,
            batch_size = 32,
            pin_memory=False
        )

        #DEFINO EL DICCIONARIO PARA CORRER
        #=================================TRAIN=================================

        info_run = {
            'target_shape': (12, 32, 32),
            'sequence_embedding_features': 18432,
            'weights_path' : f'/data/ExperimentsPercentTRIPLETBaseline/Experiments_with_80.0%/models/mertash_contrastive_fold_{k+1}.pt',
            'data' : train_loader,
                    
            }

        embeddings_train, labels_train = generate_vectors(info_run)

        print(f"Embeddings train shape{embeddings_train.shape}")
        print(f"Labels train shape{labels_train.shape}")

        destination_path_train = f"/data/Embeddings80%/embeddingsTriplet/fold_{k+1}/train" #CAMBIAR
        os.makedirs(destination_path_train, exist_ok=True)

        # Nombres de archivo para guardar
        embeddings_file = os.path.join(destination_path_train, "embeddings_train.pt")
        labels_file = os.path.join(destination_path_train, "labels_train.pt")

        # Guardar los tensores de embeddings y labels
        torch.save(embeddings_train, embeddings_file)
        torch.save(labels_train, labels_file)

        print(f"Embeddings guardados en: {embeddings_file}")
        print(f"Labels guardados en: {labels_file}")





        # =================================VALIDATION=================================
        info_run = {
            'target_shape': (12, 32, 32),
            'sequence_embedding_features': 18432,
            'weights_path' : f'/data/ExperimentsPercentTRIPLETBaseline/Experiments_with_80.0%/models/mertash_contrastive_fold_{k+1}.pt',
            'data' : val_loader,
                    
            }

        embeddings_val, labels_val = generate_vectors(info_run)

        destination_path_val = f"/data/Embeddings80%/embeddingsTriplet/fold_{k+1}/val" #CAMBIAR
        os.makedirs(destination_path_val, exist_ok=True)

        # Nombres de archivo para guardar
        embeddings_file = os.path.join(destination_path_val, "embeddings_val.pt")
        labels_file = os.path.join(destination_path_val, "labels_val.pt")

        # Guardar los tensores de embeddings y labels
        torch.save(embeddings_val, embeddings_file)
        torch.save(labels_val, labels_file)

        print(f"Embeddings guardados en: {embeddings_file}")
        print(f"Labels guardados en: {labels_file}")





-------Cargando los datos del JSON revuelto con el 80.0 % en el fold 1-------
0.8
X_train shape: (828, 3, 12, 32, 32)
Y_train shape: (828,)
X_val shape: (260, 3, 12, 32, 32)
Y_val shape: (260,)
26
torch.Size([828, 1])
torch.Size([828, 96, 96])
torch.Size([96, 96])
Embeddings train shapetorch.Size([828, 4560])
Labels train shapetorch.Size([828, 1])
Embeddings guardados en: /data/Embeddings80%/embeddingsTriplet/fold_1/train/embeddings_train.pt
Labels guardados en: /data/Embeddings80%/embeddingsTriplet/fold_1/train/labels_train.pt
9
torch.Size([260, 1])
torch.Size([260, 96, 96])
torch.Size([96, 96])
Embeddings guardados en: /data/Embeddings80%/embeddingsTriplet/fold_1/val/embeddings_val.pt
Labels guardados en: /data/Embeddings80%/embeddingsTriplet/fold_1/val/labels_val.pt
-------Cargando los datos del JSON revuelto con el 80.0 % en el fold 2-------
0.8
X_train shape: (824, 3, 12, 32, 32)
Y_train shape: (824,)
X_val shape: (264, 3, 12, 32, 32)
Y_val shape: (264,)
26
torch.Size([824, 1])
to

In [38]:
i=5


train_embeddings_path = f'/data/Embeddings80%/embeddingsTriplet/fold_{i}/train/embeddings_train.pt'
train_labels_path = f'/data/Embeddings80%/embeddingsTriplet/fold_{i}/train/labels_train.pt'

train_embeddings = torch.load(train_embeddings_path)
train_labels = torch.load(train_labels_path)

val_embeddings_path = f'/data/Embeddings80%/embeddingsTriplet/fold_{i}/val/embeddings_val.pt'
val_labels_path = f'/data/Embeddings80%/embeddingsTriplet/fold_{i}/val/labels_val.pt'

val_embeddings = torch.load(val_embeddings_path)
val_labels = torch.load(val_labels_path)



print(train_embeddings.shape)
print(train_labels.shape)
print('')
print(val_embeddings.shape)
print(val_labels.shape)



torch.Size([834, 4560])
torch.Size([834, 1])

torch.Size([252, 4560])
torch.Size([252, 1])


In [1]:
import json
fold_indexes = open('/data/picai_folds_indexes.json', 'r')
indexdes = json.load(fold_indexes)

#Imprimir el tamanńo de los datos de validación y entrenamiento por fold
for i in range(5):
    print(len(indexdes[f'Fold_{i}_train']) + len(indexdes[f'Fold_{i}_val']))
    # print(len(indexdes[f'Fold_{i}_val']))


# array = []

# for i in range (5):
#     array.append(len(indexdes[f'Fold_{i}_val']))
    
# print(np.sum(array))

1295
1295
1295
1295
1295


In [4]:
import json
fold_indexes = open('/data/picai_folds_indexes.json', 'r')
indexdes = json.load(fold_indexes)

len(indexdes['Fold_0_train'])

1035

# Logistic Regression

In [43]:
import torch
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, roc_auc_score, precision_recall_curve, auc

array_auc = []
array_accuracy = []
array_auc_pr = []

for i in range(5):
    # Cargar embeddings y etiquetas de entrenamiento
    embeddings_train = torch.load(f'/data/Embeddings100%/embeddings/fold_{i+1}/train/embeddings_train.pt')
    labels_train = torch.load(f'/data/Embeddings100%/embeddings/fold_{i+1}/train/labels_train.pt')

    # Cargar embeddings y etiquetas de validación
    embeddings_val = torch.load(f'/data/Embeddings100%/embeddings/fold_{i+1}/val/embeddings_val.pt')
    labels_val = torch.load(f'/data/Embeddings100%/embeddings/fold_{i+1}/val/labels_val.pt')

    # Convertir a numpy arrays
    embeddings_train_np = embeddings_train.cpu().numpy()
    labels_train_np = labels_train.cpu().numpy()

    embeddings_val_np = embeddings_val.cpu().numpy()
    labels_val_np = labels_val.cpu().numpy()

    # Entrenar el modelo de regresión logística
    model = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000, random_state=42))
    model.fit(embeddings_train_np, labels_train_np)

    # Predicciones en el conjunto de validación
    predictions = model.predict(embeddings_val_np)
    predictions_proba = model.predict_proba(embeddings_val_np)[:, 1]

    # Calcular la exactitud
    accuracy = accuracy_score(labels_val_np, predictions)
    print(f'Accuracy: {accuracy:.4f}')
    array_accuracy.append(accuracy)

    # Calcular el AUC-ROC
    auc_roc = roc_auc_score(labels_val_np, predictions_proba)
    print(f'AUC-ROC: {auc_roc:.4f}')
    array_auc.append(auc_roc)

    # Calcular el AUC-PR
    precisions, recalls, _  = precision_recall_curve(labels_val_np, predictions_proba)
    auc_pr = auc(recalls, precisions)
    print(f'AUC-PR: {auc_pr:.4f}')
    
    array_auc_pr.append(auc_pr)

print(f"Promedio de AUC-ROC: {np.mean(array_auc)}")
print(f"Desviación de AUC-ROC: {np.std(array_auc)}")
print("")
print(f"Promedio de Accuracy: {np.mean(array_accuracy)}")
print(f"Desviación de Accuracy: {np.std(array_accuracy)}")
print("")
print(f"Promedio de AUC-PR: {np.mean(array_auc_pr)}")
print(f"Desviación de AUC-PR: {np.std(array_auc_pr)}")


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.9077
AUC-ROC: 0.8849
AUC-PR: 0.7924


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.9205
AUC-ROC: 0.9046
AUC-PR: 0.8178


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.9213
AUC-ROC: 0.9373
AUC-PR: 0.8129


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.9170
AUC-ROC: 0.9013
AUC-PR: 0.7925


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.9127
AUC-ROC: 0.9312
AUC-PR: 0.8076
Promedio de AUC-ROC: 0.9118438368053112
Desviación de AUC-ROC: 0.019545317235410302

Promedio de Accuracy: 0.9158172480880845
Desviación de Accuracy: 0.0050640453704674975

Promedio de AUC-PR: 0.8046268951211847
Desviación de AUC-PR: 0.010469080045039356


# SVM

In [1]:
import torch
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, roc_auc_score, average_precision_score

array_auc = []
array_accuracy = []
array_auc_pr = []

for i in range(5):
    # Cargar embeddings y etiquetas de entrenamiento
    embeddings_train = torch.load(f'/data/Embeddings80%/embeddingsTriplet/fold_{i+1}/train/embeddings_train.pt')
    labels_train = torch.load(f'/data/Embeddings80%/embeddingsTriplet/fold_{i+1}/train/labels_train.pt')

    # Cargar embeddings y etiquetas de validación
    embeddings_val = torch.load(f'/data/Embeddings80%/embeddingsTriplet/fold_{i+1}/val/embeddings_val.pt')
    labels_val = torch.load(f'/data/Embeddings80%/embeddingsTriplet/fold_{i+1}/val/labels_val.pt')

    # Convertir a numpy arrays
    embeddings_train_np = embeddings_train.cpu().numpy()
    labels_train_np = labels_train.cpu().numpy()

    embeddings_val_np = embeddings_val.cpu().numpy()
    labels_val_np = labels_val.cpu().numpy()

    # Entrenar el modelo de SVM
    # model = make_pipeline(StandardScaler(), SVC(probability=True, random_state=42))
    model = make_pipeline(
    StandardScaler(), 
    SVC(kernel='rbf', probability=True, random_state=42)
)
    model.fit(embeddings_train_np, labels_train_np)

    # Predicciones en el conjunto de validación
    predictions = model.predict(embeddings_val_np)
    predictions_proba = model.predict_proba(embeddings_val_np)[:, 1]

    # Calcular la exactitud
    accuracy = accuracy_score(labels_val_np, predictions)
    print(f'Accuracy: {accuracy:.4f}')
    array_accuracy.append(accuracy)

    # Calcular el AUC-ROC
    auc_roc = roc_auc_score(labels_val_np, predictions_proba)
    print(f'AUC-ROC: {auc_roc:.4f}')
    array_auc.append(auc_roc)

    # Calcular el AUC-PR
    auc_pr = average_precision_score(labels_val_np, predictions_proba)
    print(f'AUC-PR: {auc_pr:.4f}')
    array_auc_pr.append(auc_pr)

print(f"Promedio de AUC-ROC: {np.mean(array_auc)}")
print(f"Desviación de AUC-ROC: {np.std(array_auc)}")
print("")
print(f"Promedio de Accuracy: {np.mean(array_accuracy)}")
print(f"Desviación de Accuracy: {np.std(array_accuracy)}")
print("")
print(f"Promedio de AUC-PR: {np.mean(array_auc_pr)}")
print(f"Desviación de AUC-PR: {np.std(array_auc_pr)}")


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.9038
AUC-ROC: 0.8807
AUC-PR: 0.7786


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.9318
AUC-ROC: 0.9006
AUC-PR: 0.8388


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.9055
AUC-ROC: 0.9215
AUC-PR: 0.7942


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.9019
AUC-ROC: 0.9491
AUC-PR: 0.8519


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.8929
AUC-ROC: 0.9385
AUC-PR: 0.8319
Promedio de AUC-ROC: 0.9180610938482205
Desviación de AUC-ROC: 0.024880892452682973

Promedio de Accuracy: 0.9071840163995862
Desviación de Accuracy: 0.013069809410691389

Promedio de AUC-PR: 0.8190740193510881
Desviación de AUC-PR: 0.027856818562675127


1
